In [1]:
# Crawler Package
import urllib.request as req
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

# Date and Time Package
from dateutil import rrule
from datetime import date
from datetime import datetime
from datetime import timedelta
from dateutil import relativedelta

# Data Preprocessing Package
import re
import pandas as pd
import os
import openpyxl
import random
import csv

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### Define Path

In [2]:
"""
기본적인 경로 정의
본인의 환경에 맞춰 경로를 정의하면 됨
"""

ROOT_PATH = "./Text_Mining/" # 기본 저장 주소
DATA_FOLDER_PATH = os.path.join(ROOT_PATH,"Results/") # 폴더명 
DATA_FILE_NAME = os.path.join(DATA_FOLDER_PATH,"contents.xlsx") # 이름을 설정하세요
LINK_FILE_NAME = os.path.join(DATA_FOLDER_PATH,"links.xlsx") # 이름을 설정하세요

options = Options()

user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
options.add_argument('user-agent=' + user_agent)

options.add_argument('--headless')
options.add_argument('--no-sadnbox')
options.add_argument('--disable-dev-shm-usage')

service = Service(executable_path='C:/Users/jrack/chromedriver.exe') # 크롬드라이버 저장 위치로 변경

options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches',['enable-logging'])

driver = webdriver.Chrome(service=service,options=options) # 크롬드라이버 실행. 아무것도 안 뜨는 게 정상임

"""
상세검색 적용을 위한 네이버 뉴스 URL 정의
"""

first='https://search.naver.com/search.naver?where=news&query='
second='&sm=tab_opt&sort='
third='&photo=0&field=0&pd=3&ds='
fourth='&de='
fifth='&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom'
sixth='to'
seventh='&is_sug_officeid=0&office_category=0&service_area=0'

In [8]:
"""
검색어 및 검색조건 튜닝
"""
'''
 네이버 키워드 검색은 여러가지 옵션이 있습니다. <- 이 부분은 추가로 확인이 필요함
 
 1. + : +는 단어를 추가하여 검색 결과를 좁히는 옵션 
 "손흥민 + 토트넘" 으로 키워드를 입력하면, 
 손흥민 검색 결과 중, 토트넘을 포함한 상세 검색이 실시 
 주의!!! 본 크롤러에서는 +를 사용한다면 %2b로 입력
 
 2. - : -는 단어를 제외하여 검색 결과를 좁히는 옵션
 주의!!! 본 크롤러에서는 -를 사용한다면 -로 입력
 
 3. "" : ""큰 따옴표는 ""안에 들어가는 정확한 키워드를 포함하는 검색을 실시 
 
 ex)손흥민 "베일" +토트넘
 -->'손흥민'에 대한 검색결과 중 "베일"이 정확하게 일치하고 '토트넘'을 포함한 상세검색 결과
 
여러개의 키워드를 입력할시, 리스트 형태로 ex) ['손흥민', '토트넘', '베일 ']
 
'''
#1번째 키워드: ['손흥민 "무리뉴" %2b토트넘']
#'손흥민'에 대한 검색결과 중 "무리뉴"이 정확하게 일치하고 '토트넘'을 포함한 상세검색
#2번째 키워드: [아스널 "아르테타" -외질']
#'아스널'에 대한 검색결과 중 "아르테타"가 정확하게 일치하고 '외질'을 제외한 상세검색 결과입니다.
# keyword_list = ['손흥민 "무리뉴" %2b토트넘','아스널 "아르테타" -외질'] 

keyword_list = ['동작갑'] 
sort = '0'         # 정렬방식 : 0 관련도순, 1 최신 순, 2 오래된 순
#pattern = '0'       # 검색유형 설정: 0 전체, 1 제목
start = '20240227'   # 시작 날짜
last = '20240227'    # 종료 날짜
start_1='2024.02.27'
last_1='2024.02.27' 

In [9]:
'''
네이버뉴스 링크만 수집
'''

spt = 0.5

href_link = [] #링크 모아두는 리스트
error_link = [] # 에러가 나는 링크 모아두는 리스트 

for i in keyword_list:
    keyword = i
    query=first+keyword+second+sort+third+start_1+fourth+last_1+fifth+start+sixth+last+seventh
    driver.get(query)
    last_height = driver.execute_script("return document.body.scrollHeight")
  
        #i=0
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 처음 화면 끝까지 스크롤
        time.sleep(spt)
        new_height = driver.execute_script("return document.body.scrollHeight") # 스크롤했을 때 높이 저장
        if new_height == last_height: # 스크롤한 창의 높이가 가장 하단 창의 높이와 같아지면 break
            break
        last_height = new_height
        

    html = driver.page_source
    soup = bs(html, 'html.parser')
    a_tags=soup.find_all("a",{'class': "info"})
    for a_tag in a_tags:
        tmp=a_tag["href"]
        if 'naver' in tmp: # 링크에 naver가 포함된 경우만 추가
            href_link.append(tmp)

print(len(href_link))

        

# 중복 제거 후 리스트화
href_link=set(href_link)
href_link=list(href_link)
print(len(href_link))


df_link= pd.DataFrame(href_link)
df_link.to_excel(LINK_FILE_NAME, sheet_name='sheet1') # 해당 파일 또는 링크는 뉴스 본문 크롤링 시 사용 가능

8
7


C:\Users\jrack\AppData\Local\Temp\ipykernel_29244\2256813565.py:45: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df_link.to_excel(LINK_FILE_NAME, sheet_name='sheet1') # 해당 파일 또는 링크는 뉴스 본문 크롤링 시 사용 가능


In [10]:
"""
댓글 페이지 주소 생성
"""

cmts_links=[]

for i in range(len(href_link)):
    cmts_link = href_link[i][:39]+'comment/'+href_link[i][39:] # 댓글 페이지 주소 생성
    cmts_links.append(cmts_link)

In [11]:
news_comments = []

for i in range(len(cmts_links)):
    driver.get(cmts_links[i])
    time.sleep(2)
    # 더보기 버튼이 더이상 안 나올 때까지 클릭
    try:
        for j in range(1000000):
            btn=driver.find_element(By.XPATH, '//*[@id="cbox_module"]/div[2]/div[9]/a/span/span/span[1]')
            btn.click() # 더보기 버튼 클릭
    except:
        print('더보기 버튼 없음')
    
    html = driver.page_source
    soup_1 = bs(html, 'html.parser')
    
    title=soup_1.find('title') # 기사 타이틀 가져오기
    cmmnts = soup_1.find_all("div", {'class':'u_cbox_text_wrap'}) # 댓글들 내용 가져오기
    cmmnts_date = soup_1.find_all('span', {'class':'u_cbox_date'}) # 댓글 단 날짜들 가져오기
        
    for k in range(len(cmmnts)):
        comment=[]
        comment.append(title.get_text())
        comment.append(cmmnts_date[k].get_text())
        comment.append(cmmnts[k].get_text())
        news_comments.append(comment) # 댓글 하나마다 기사 타이틀, 댓글 단 날짜, 댓글 내용 순서로 추가

news_cmmnts = pd.DataFrame(news_comments, columns = ['title', 'cmmnt_date', 'comment'])
news_cmmnts = news_cmmnts.set_index('title') # index 설정
news_cmmnts['comment']=news_cmmnts['comment'].str.replace('내가 차단한 이용자의 댓글입니다.','') # 필요없는 내용 제거
news_cmmnts.to_excel('news_cmmnts_dongjak_gap.xlsx') # 엑셀로 저장

더보기 버튼 없음
더보기 버튼 없음
더보기 버튼 없음
더보기 버튼 없음
더보기 버튼 없음
더보기 버튼 없음
더보기 버튼 없음


C:\Users\jrack\AppData\Local\Temp\ipykernel_29244\2518273860.py:31: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  news_cmmnts.to_excel('news_cmmnts_dongjak_gap.xlsx')


In [22]:
title_grouped = news_cmmnts.groupby('title')['comment'].apply(lambda x: ';'.join(x))

In [24]:
title_grouped.to_excel('total_comments.xlsx')

C:\Users\jrack\AppData\Local\Temp\ipykernel_29244\1326803079.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df_grouped.to_excel('total_comments.xlsx')


In [32]:
pd.DataFrame(title_grouped)

,comment
title,
[사설] 민주당 후보검증위장 불법자금 의혹 사실인가,그간 그들의 비 도덕적 행태를보면 안한게 더 이상한 집단인듯;사실인가봐 ᆢㅋ...
與 '공천 보류' 63곳 고심…험지 '인재 배치'·양지 '잡음 최소화' 과제,작성자에 의해 삭제된 댓글입니다.
고삐 풀린 文·明충돌···민주당 공천 내홍 ‘격화’ 국힘 1년만에 지지율 역전,국힘당 공천으로 자살 소동도 있고 국힘당 당사 앞 삭발도 하던데 그건 안 보이나? ...
"국민의힘, 험지 빈집 채우기에 고심…텃밭은 공천잡음 우려",클린봇이 부적절한 표현을 감지한 댓글입니다.;ㅋㅋㅋ김거니 재표결 끝나면 칼부림 나겠...
"민주당 김병기, 이수진 고소…전여옥 ""이런 막장은 처음""",진짜 민주당은 막장이다. ㅠ국민들이여 민주당 심판해주세요;그동안 나쁜놈들에게 속고...
"의료현장 번아웃 계속…뉴욕증시, 약보합 마감 [모닝브리핑]",술먹은꽐라 경증환자만 응급실 안받아도 뺑뺑이 해결 훨씬된다 .. 2천명 건보료폭탄만...
"전여옥, 이수진에 엄지 척 “갑자기 응원하고 싶어…‘재명808’ 이재명 겪고 술이 확 깨”",클린봇이 부적절한 표현을 감지한 댓글입니다.;자기가 싫은사람은 다모지리라고 생각하는...
